In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm

In [ ]:
df = pd.read_csv('Abandono_Clientes_Bancarios.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
dummy_gender = pd.get_dummies(df["Gender"], prefix="Gender")
dummy_geography = pd.get_dummies(df["Geography"], prefix="Geography")
df1 = df.drop(["Surname", "CustomerId","Tenure","RowNumber"], axis=1)
df2 = df1.drop(["Geography"], axis=1)
df3 = df2.drop(["Gender"], axis=1)

In [ ]:
df3 = pd.concat([df3, dummy_gender, dummy_geography], axis=1)
df3.head()

,CreditScore,Age,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,0.00,1,1,1,101348.88,1,1,0,1,0,0
1,608,41,83807.86,1,0,1,112542.58,0,1,0,0,0,1
2,502,42,159660.80,3,1,0,113931.57,1,1,0,1,0,0
3,699,39,0.00,2,0,0,93826.63,0,1,0,1,0,0
4,850,43,125510.82,1,1,1,79084.10,0,1,0,0,0,1


In [ ]:
df3 = df3[['CreditScore','Age','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Gender_Male','Geography_France','Geography_Spain','Exited']]
df3.head()

,CreditScore,Age,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_France,Geography_Spain,Exited
0,619,42,0.00,1,1,1,101348.88,0,1,0,1
1,608,41,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,159660.80,3,1,0,113931.57,0,1,0,1
3,699,39,0.00,2,0,0,93826.63,0,1,0,0
4,850,43,125510.82,1,1,1,79084.10,0,0,1,0


Decidimos omitir las variables "Gender_Female" y "Geography_Germany" para no causar una dependencia entre variables

In [ ]:
X = df3.iloc[:, :10]
y = df3.iloc[:, -1]

In [ ]:
import sklearn.ensemble as RandomForestClassifier
import sklearn.model_selection as cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state=0)
Xtrain.shape, Xtest.shape

((8000, 10), (2000, 10))

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=2)
for valores_x, valores_y in kf.split(X):
  print("Entrenamiento: ", df3.iloc[valores_x], "Prueba: ", df3.iloc[valores_y])

Entrenamiento:        CreditScore  Age    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0             619   42       0.00              1          1               1   
1             608   41   83807.86              1          0               1   
2             502   42  159660.80              3          1               0   
5             645   44  113755.78              2          1               0   
6             822   50       0.00              2          1               1   
...           ...  ...        ...            ...        ...             ...   
9992          726   36       0.00              1          1               0   
9995          771   39       0.00              2          1               0   
9996          516   35   57369.61              1          1               1   
9997          709   36       0.00              1          0               1   
9998          772   42   75075.31              2          1               0   

      EstimatedSalary  Gender_Male 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state=0)
Xtrain.shape, Xtest.shape

((8000, 10), (2000, 10))

In [ ]:
lr = LogisticRegression()
lr.fit(Xtrain, ytrain)
lr.score(Xtest, ytest)

0.789

In [ ]:
svm = SVC()
svm.fit(Xtrain, ytrain)
svm.score(Xtest, ytest)

0.7975

In [ ]:
rf = RandomForestClassifier(n_estimators=30)
rf.fit(Xtrain, ytrain)
rf.score(Xtest, ytest)

0.8525

No podemos expresar que un modelo es mejor que otro dado que el score de los mismos pueden cambiar dependiendo la cantidad de veces que entrenemos el modelo ya que lo hace uniformemente. Es por esto que ahora usaremos KFold para verificar que tan bueno es nuestro algoritmo y asi determinar si hay Overfitting o Underfitting. 

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [ ]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
  print(train_index, test_index)

[2 3 4 5 6 7 8] [0 1]
[0 1 4 5 6 7 8] [2 3]
[0 1 2 3 6 7 8] [4 5]
[0 1 2 3 4 5 8] [6 7]
[0 1 2 3 4 5 6 7] [8]


In [ ]:
def get_score(model, Xtrain, Xtest, ytrain, ytest):
  model.fit(Xtrain, ytrain)
  return model.score(Xtest, ytest)

In [ ]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(LogisticRegression(), X, y).mean()

0.7902

In [ ]:
lr = LogisticRegression()
lr.fit(Xtrain, ytrain)
lr.score(Xtest, ytest)

0.789

Los valores son muy similares por lo que no estaríamos en una situación de overfitting o underfitting.

In [ ]:
cross_val_score(SVC(), X, y).mean()

0.7963

In [ ]:
svm = SVC()
svm.fit(Xtrain, ytrain)
svm.score(Xtest, ytest)

0.7975

Los valores son muy similares por lo que no estaríamos en una situación de overfitting o underfitting.

In [ ]:
cross_val_score(RandomForestClassifier(), X, y).mean()

0.8594000000000002

In [ ]:
rf = RandomForestClassifier(n_estimators=30)
rf.fit(Xtrain, ytrain)
rf.score(Xtest, ytest)

0.8605

Los valores son muy similares por lo que no estaríamos en una situación de overfitting o underfitting.

En casi todos los modelos que evaluamos, el cross validation estuvo un poquito por debajo que los valores esperados, lo que nos podría estar diciendo que hay una situación mínima de underfitting. Para prevenir esta situación lo que podriamos hacer es tratar de eliminar datos atípicos como tan bien eliminar variables que no nos sean de uso tan necesario, también podriamos tratar de usar modelos mas complejos para mejorar la situación de nuestro DataSet.